In [51]:
import pandas as pd
import base64
import requests as rq
import json

# 1. Headers

In [8]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

apikey=secrets_dict['apikey']
secret=secrets_dict['secret']

In [35]:
to_encode = apikey+':'+secret

In [38]:
authorization = 'Basic ' + base64.b64encode(to_encode.encode("ascii")).decode("ascii")
content_type = 'application/x-www-form-urlencoded;charset=UTF-8'

headers = {'Authorization' : authorization, 'Content-Type' : content_type}

# 2. Token

In [40]:
parameters = {'grant_type' : 'client_credentials', 'scope' : 'read'}

In [41]:
request = rq.post("https://api.idealista.com/oauth/token",  headers = headers, params = parameters)

In [42]:
request

<Response [200]>

In [43]:
token = json.loads(request.text)['access_token']

# 3. URL

In [59]:
operation = 'sale'
max_items = '50'
order = 'priceDown'
# center = '43.26271, -2.92528'
# distance = '60000'
locationId = '0-EU-ES-48-01-001-020'
sort = 'desc'
property_type = 'homes'
maxprice = '450000'
language = 'es' 
minSize = '40'
bankOffer = 'False'


In [92]:
url = ('https://api.idealista.com/3.5/es' +
       '/search?operation=' + str(operation) +
       '&maxItems=' + str(max_items) +
       '&order=' + str(order) +
       '&locationId=' + str(locationId) +
       '&sort=' + str(sort) +
       '&propertyType=' + str(property_type) +
       '&numPage=%s' +
       '&maxPrice=' + str(maxprice) +
       '&language=' + str(language) +
       '&minSize=' + str(minSize) +
       '&bankOffer=' + str(bankOffer))

# 4. Request Content

In [61]:
def request_content(url):  


    headers = {'Content-Type': 'Content-Type: multipart/form-data;',
               'Authorization' : 'Bearer ' + token}

    content = rq.post(url, headers = headers)

    result = json.loads(content.text)

    return result

In [62]:
pagination = 1
first_search_url = url %(pagination)

In [63]:
results = request_content(first_search_url)

In [64]:
results

{'elementList': [{'propertyCode': '100138386',
   'thumbnail': 'https://img3.idealista.com/blur/WEB_LISTING/0/id.pro.es.image.master/9f/ef/29/1070502799.jpg',
   'numPhotos': 61,
   'floor': '5',
   'price': 175000.0,
   'propertyType': 'flat',
   'operation': 'sale',
   'size': 75.0,
   'exterior': True,
   'rooms': 3,
   'bathrooms': 1,
   'address': 'Calle Aurrekoetxea',
   'province': 'Vizcaya',
   'municipality': 'Bilbao',
   'district': 'Ibaiondo',
   'country': 'es',
   'neighborhood': 'Iturralde',
   'latitude': 43.2556863,
   'longitude': -2.9140692,
   'showAddress': False,
   'url': 'https://www.idealista.com/inmueble/100138386/',
   'description': 'Se vende piso AMUEBLADO en Begoña a 5 minutos andando de la boca del metro de Santutxu con ascensor, calefacción individual gas naturaL. La vivienda se distribuye en tres dormitorios, sala, cocina equipada con acceso a un balcón cerrado y baño completo con bañera. Es exterior y muy luminoso. Yolanda:.',
   'hasVideo': False,
   '

In [65]:
total_pages = results['totalPages']

In [66]:
total_pages

40

In [68]:
df = pd.DataFrame.from_dict(results['elementList'])

In [70]:
df.shape

(50, 38)

In [72]:
df.to_csv('idealista_231009_1.csv', index=False)

In [74]:
all_data = df.copy()

In [99]:
for i in range(10, total_pages): 

    url_i = url %(i)
    results = request_content(url_i)
    df_i = pd.DataFrame.from_dict(results['elementList'])
    all_data = pd.concat([all_data, df_i])

In [100]:
all_data

,propertyCode,thumbnail,numPhotos,floor,price,propertyType,operation,size,exterior,rooms,...,has3DTour,has360,hasStaging,superTopHighlight,topNewDevelopment,externalReference,highlight,parkingSpace,labels,newDevelopmentFinished
0,100138386,https://img3.idealista.com/blur/WEB_LISTING/0/...,61,5,175000.0,flat,sale,75.0,True,3,...,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
1,28897633,https://img3.idealista.com/blur/WEB_LISTING/0/...,12,5,160000.0,flat,sale,106.0,True,3,...,False,False,False,False,False,11859,{'groupDescription': 'Destacado'},NaN,NaN,NaN
2,93866569,https://img3.idealista.com/blur/WEB_LISTING/0/...,22,3,98000.0,flat,sale,55.0,True,2,...,False,False,False,False,False,CA91,NaN,NaN,NaN,NaN
3,93696863,https://img3.idealista.com/blur/WEB_LISTING/0/...,11,3,98000.0,flat,sale,55.0,True,2,...,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
4,99918479,https://img3.idealista.com/blur/WEB_LISTING/0/...,6,2,410000.0,flat,sale,110.0,True,2,...,False,False,False,False,False,13828,{'groupDescription': 'Destacado'},NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,87453586,https://img3.idealista.com/blur/WEB_LISTING/0/...,2,NaN,430000.0,flat,sale,140.0,True,2,...,False,False,False,False,False,1061,NaN,NaN,NaN,NaN
45,99641521,https://img3.idealista.com/blur/WEB_LISTING/0/...,1,6,360000.0,flat,sale,75.0,False,2,...,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
46,101966119,NaN,0,7,270000.0,penthouse,sale,77.0,True,3,...,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
47,98157298,NaN,0,1,420000.0,flat,sale,112.0,True,3,...,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN


In [101]:
# all_data.to_csv('idealista_231009_2.csv', index=False)